In [106]:
import pandas as pd
from sklearn import metrics
from sklearn.metrics import recall_score, classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from imblearn.combine import SMOTEENN


In [107]:
df=pd.read_csv('tel_churn.csv')
df.head()

,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1-12,tenure_group_13-24,tenure_group_25-36,tenure_group_37-48,tenure_group_49-60,tenure_group_61-72
0,0,0,29.85,29.85,0,True,False,False,True,True,False,True,False,False,True,False,True,False,False,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,False,False,False,False
1,1,0,56.95,1889.50,0,False,True,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,False,True,False,True,False,False,False,False,True,False,False,True,False,False,False
2,2,0,53.85,108.15,1,False,True,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,True,True,False,False,False,False,False
3,3,0,42.30,1840.75,0,False,True,True,False,True,False,True,False,False,True,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False,True,True,False,False,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,False,False
4,4,0,70.70,151.65,1,True,False,True,False,True,False,False,True,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,False,False,False,False


In [108]:
df=df.drop(columns='Unnamed: 0',axis=1)


In [109]:
X=df.drop('Churn',axis=1)
X

,SeniorCitizen,MonthlyCharges,TotalCharges,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,MultipleLines_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1-12,tenure_group_13-24,tenure_group_25-36,tenure_group_37-48,tenure_group_49-60,tenure_group_61-72
0,0,29.85,29.85,True,False,False,True,True,False,True,False,False,True,False,True,False,False,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,False,False,False,False
1,0,56.95,1889.50,False,True,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,False,True,False,True,False,False,False,False,True,False,False,True,False,False,False
2,0,53.85,108.15,False,True,True,False,True,False,False,True,True,False,False,True,False,False,False,False,True,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,True,True,False,False,False,False,False
3,0,42.30,1840.75,False,True,True,False,True,False,True,False,False,True,False,True,False,False,False,False,True,True,False,False,False,False,True,False,False,True,True,False,False,True,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,False,False
4,0,70.70,151.65,True,False,True,False,True,False,False,True,True,False,False,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,0,84.80,1990.50,False,True,False,True,False,True,False,True,False,False,True,True,False,False,False,False,True,True,False,False,False,False,True,False,False,True,False,False,True,False,False,True,False,True,False,False,True,False,False,False,True,False,True,False,False,False,False
7028,0,103.20,7362.90,True,False,False,True,False,True,False,True,False,False,True,False,True,False,True,False,False,False,False,True,False,False,True,True,False,False,False,False,True,False,False,True,False,True,False,False,True,False,True,False,False,False,False,False,False,False,True
7029,0,29.60,346.45,True,False,False,True,False,True,True,False,False,True,False,True,False,False,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,True,False,True,False,False,False,False,False
7030,1,74.40,306.60,False,True,False,True,True,False,False,True,False,False,True,False,True,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,True,True,False,False,False,False,False


In [110]:
y=df['Churn']

In [111]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

## Decision Tree Classifier


In [112]:
model_DT=DecisionTreeClassifier(criterion='gini',random_state=100,min_samples_leaf=8,max_depth=6)

In [113]:
model_DT.fit(X_train,y_train)


DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [114]:
y_pred=model_DT.predict(X_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [115]:
model_DT.score(X_test,y_test)

0.7846481876332623

In [116]:
print(classification_report(y_test,y_pred,labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      1014
           1       0.63      0.55      0.59       393

    accuracy                           0.78      1407
   macro avg       0.73      0.71      0.72      1407
weighted avg       0.78      0.78      0.78      1407



As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

In [117]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(X, y)  


In [118]:
Xr_train,Xr_test,yr_train,yr_test=train_test_split(X_resampled,y_resampled,test_size=0.2)

In [119]:
model_DT_smote=DecisionTreeClassifier(criterion='gini',max_depth=6,min_samples_leaf=8,random_state=100)

In [120]:
model_DT_smote.fit(Xr_train,yr_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [121]:
yr_pred=model_DT_smote.predict(Xr_test)

In [130]:
model_score_r=model_DT_smote.score(Xr_test,yr_test)

In [131]:
print(model_score_r)

0.9222886421861657


In [124]:
print(metrics.classification_report(yr_test, yr_pred))


              precision    recall  f1-score   support

           0       0.92      0.91      0.91       523
           1       0.93      0.94      0.93       648

    accuracy                           0.92      1171
   macro avg       0.92      0.92      0.92      1171
weighted avg       0.92      0.92      0.92      1171



**Accuracy:** 0.93 → The model correctly predicted 93% of all instances.


In [125]:
print(metrics.confusion_matrix(yr_test,yr_pred))

[[474  49]
 [ 42 606]]


Now we can see quite better results, a very good recall, precision & f1 score for minority class.

Let's try with some other classifier.

## Random Forest Classifier


In [126]:
model_rf=RandomForestClassifier(criterion='gini',n_estimators=100,random_state=100,max_depth=6,min_samples_leaf=8)

In [128]:
model_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [129]:
y_pred=model_rf.predict(X_test)

In [132]:
model_rf.score(X_test,y_test)

0.7839374555792467

In [133]:
print(classification_report(y_test, y_pred, labels=[0,1]))


              precision    recall  f1-score   support

           0       0.81      0.92      0.86      1014
           1       0.67      0.44      0.53       393

    accuracy                           0.78      1407
   macro avg       0.74      0.68      0.70      1407
weighted avg       0.77      0.78      0.77      1407



In [134]:
sm=SMOTEENN()
X_resampled1,y_resampled1=sm.fit_resample(X,y)

In [139]:
Xr_train1,Xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)


In [140]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)


In [141]:
model_rf_smote.fit(Xr_train1,yr_train1)


RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [142]:
yr_predict1 = model_rf_smote.predict(Xr_test1)


In [143]:
model_score_r1 = model_rf_smote.score(Xr_test1, yr_test1)


In [144]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9341317365269461
              precision    recall  f1-score   support

           0       0.96      0.89      0.92       521
           1       0.92      0.97      0.94       648

    accuracy                           0.93      1169
   macro avg       0.94      0.93      0.93      1169
weighted avg       0.94      0.93      0.93      1169



In [145]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))


[[464  57]
 [ 20 628]]


With RF Classifier, also we are able to get quite good results, infact better than Decision Tree.


## Performing PCA


In [156]:
# Applying PCA

from sklearn.decomposition import PCA
pca = PCA(0.9)
Xr_train_pca=pca.fit_transform(Xr_train1)
Xr_test_pca=pca.transform(Xr_test1)
explained_variance=pca.explained_variance_ratio_

In [157]:
model=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)


In [158]:
model.fit(Xr_train_pca,yr_train1)


RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [159]:
yr_predict_pca = model.predict(Xr_test_pca)


In [160]:
model_score_r_pca = model.score(xr_test_pca, yr_test1)


In [161]:
print(model_score_r_pca)
print(metrics.classification_report(yr_test1, yr_predict_pca))

0.7245508982035929
              precision    recall  f1-score   support

           0       0.70      0.68      0.69       521
           1       0.75      0.76      0.75       648

    accuracy                           0.72      1169
   macro avg       0.72      0.72      0.72      1169
weighted avg       0.72      0.72      0.72      1169



With PCA, we couldn't see any better results, hence let's finalise the model which was created by RF Classifier, and save the model so that we can use it in a later stage :)

### Pickling the model


In [162]:
import pickle

In [163]:
filname='model.sav'

In [164]:
pickle.dump(model_rf_smote,open(filname,'wb'))

In [165]:
load_model=pickle.load(open(filname,'rb'))

In [166]:
model_score_r1=load_model.score(Xr_test1,yr_test1)

In [167]:
model_score_r1

0.9341317365269461

Our final model i.e. RF Classifier with SMOTEENN, is now ready and dumped in model.sav, which we will use and prepare API's so that we can access our model from UI.